In [1]:
%load_ext autoreload
%autoreload 2

## Generating summaries

In [2]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split, get_sample_weights, get_eval_set
from src.preprocessing import preprocess_data
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, AutoModelForCausalLM
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, roc_auc_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from peft import get_peft_model, LoraConfig, TaskType
from collections import defaultdict
import transformers
from peft import get_peft_model, LoraConfig, TaskType
import re
from bert_score import BERTScorer
import langid
from src.utils import aggregate_samples, evaluate_model, compute_class_weights, remove_hashtag_links, get_first_texts
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BitsAndBytesConfig

tqdm.pandas()

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("generated_predictions_final.csv")

In [8]:
df['y_pred'] = df['summary'].str.lower().str.contains('yes')

In [22]:
print(df.query("y_pred == 1 & label == 0")['summary'].iloc[3])



**Summary of Events:**
The tweets suggest that Honduras scored a goal against Switzerland, with the score being 1-0 in favor of Switzerland at the end of the minute. Fans are excited and optimistic about the match, with some expressing hope for a win against Brazil in the next match.

**Specific Event Analysis:**
- Goal: **Yes**, scored by Honduras (not explicitly mentioned, but implied by the tweets).
- Yellow Card: **Not mentioned**
- Red Card: **Not mentioned**
- Kickoff: **Not mentioned** (no indication of a kickoff, as the match has already started)
- Halftime: **Not mentioned**
- Fulltime: **Not mentioned**


In [23]:
df

,Unnamed: 0,summary,label,Tweet,MatchID,PeriodID,y_pred
0,0,\n\n**Summary of Events:**\nThe tweets appear ...,0.0,Robbie Fowler predicting a Honduras win? #Worl...,0,0,False
1,1,\n\n**Summary of Events:**\nThe tweets appear ...,0.0,can #hon please win...\nLego Honduras and Fran...,0,1,False
2,2,\n\n**Summary of Events:**\nThe tweets suggest...,0.0,Hoy golea Costly. #HON\nMy Predictions: \n#ECU...,0,2,False
3,3,\n\n**Summary of Events:**\nThe tweets suggest...,0.0,Honduras better kick some ass\nCome on Hondura...,0,3,False
4,4,\n\n**Summary of Events:**\nThe tweets suggest...,0.0,Honduras is taking the win lets go!!\nHonduras...,0,4,False
...,...,...,...,...,...,...,...
2132,2132,\n\n**Summary of Events:**\nThe tweets suggest...,1.0,What a win!\n#MEX #win\nWe got this #MEX win!!...,19,125,True
2133,2133,\n\n**Summary of Events:**\nThe tweets suggest...,1.0,He'll yea 3-1 #MEX\n#mex doing work #ochoashow...,19,126,True
2134,2134,\n\n**Summary of Events:**\nThe tweets suggest...,1.0,FULL TIME!!!\n#CRO 1-3 #MEX\n#WorldCup\nCielit...,19,127,True
2135,2135,\n\n**Summary of Events:**\nThe tweets suggest...,1.0,Great win for #mex\n#mex for the win\n#NED vs ...,19,128,True


In [ ]:
df.groupby("MatchID").apply(lambda x : accuracy_score(x['label'], x['y_pred']))

/tmp/ipykernel_936592/57165034.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("MatchID").apply(lambda x : accuracy_score(x['label'], x['y_pred'])).mean()


0.6469408421576401